## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals, param_to_list, list_to_param

# Mute warnings:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ax.plot.trace import optimization_trace_single_method

In [3]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [4]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [5]:
Tag_client = AxClient()

[INFO 04-10 16:08:42] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [6]:
# Define bounds for each parameter type
int_bounds = {
    "z_sh": [6, 12], #NOTE - z_sh is actually half z_s (which must be even)
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-1, 1],
    "x_r2": [-1., 1.],
    # Cl is fixed as : 0.10-0.40
}

paper_vals = [6, 81, .476, 1.0]

# Number of copies for each parameter
num_copies = 4

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (num_copies-1-i), upper - i],  # adjust bounds to deal with order
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (.1*(num_copies-1-i)), upper - .1*i],  # adjust bounds to deal with order
            "value_type": "float",
        })

# Add int / float parameters that make sure 

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Cross_term_gear_ratio_product": ObjectiveProperties(minimize=False)},
    parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-10 16:08:42] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[71, 84]), RangeParameter(name='z_r2_3', parameter_type=INT, range=[70, 83]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.7, 1.0]), RangeParameter(name='x_s_1', parameter_type=FLOAT, range=[-0.8, 0.9]), RangeParameter(name='x_s_2', parameter_type=FLOAT, range=[-0.9, 0.8]), RangeParameter(name='x_s_3', parameter_type=FLOAT, range=[-1.0, 0.7]), RangeParameter(name='x_r2', parameter_type=FLOAT, range=[-0.7, 1.0]), RangeParameter(name='

In [7]:
max_cross_terms = 4**(len(int_bounds) + len(float_bounds)+1) # plus Cl.
print(f'max_cross_terms: {max_cross_terms}')

max_cross_terms: 1024


In [8]:
# -1.0*z_p2 + 1.0*z_p2_1 <= -1.0
# -1.0*x_p2_1 + 1.0*x_p2 <= -0.01

In [9]:
parameters

[{'name': 'z_sh', 'type': 'range', 'bounds': [9, 12], 'value_type': 'int'},
 {'name': 'z_sh_1', 'type': 'range', 'bounds': [8, 11], 'value_type': 'int'},
 {'name': 'z_sh_2', 'type': 'range', 'bounds': [7, 10], 'value_type': 'int'},
 {'name': 'z_sh_3', 'type': 'range', 'bounds': [6, 9], 'value_type': 'int'},
 {'name': 'z_r2', 'type': 'range', 'bounds': [73, 86], 'value_type': 'int'},
 {'name': 'z_r2_1', 'type': 'range', 'bounds': [72, 85], 'value_type': 'int'},
 {'name': 'z_r2_2', 'type': 'range', 'bounds': [71, 84], 'value_type': 'int'},
 {'name': 'z_r2_3', 'type': 'range', 'bounds': [70, 83], 'value_type': 'int'},
 {'name': 'x_s',
  'type': 'range',
  'bounds': [-0.7, 1.0],
  'value_type': 'float'},
 {'name': 'x_s_1',
  'type': 'range',
  'bounds': [-0.8, 0.9],
  'value_type': 'float'},
 {'name': 'x_s_2',
  'type': 'range',
  'bounds': [-0.9, 0.8],
  'value_type': 'float'},
 {'name': 'x_s_3',
  'type': 'range',
  'bounds': [-1.0, 0.7],
  'value_type': 'float'},
 {'name': 'x_r2',
  'ty

In [10]:
parameter_constraints

['z_sh - z_sh_1 >= 1',
 'z_sh_1 - z_sh_2 >= 1',
 'z_sh_2 - z_sh_3 >= 1',
 'z_r2 - z_r2_1 >= 1',
 'z_r2_1 - z_r2_2 >= 1',
 'z_r2_2 - z_r2_3 >= 1',
 'x_s - x_s_1 >= 0.1',
 'x_s_1 - x_s_2 >= 0.1',
 'x_s_2 - x_s_3 >= 0.1',
 'x_r2 - x_r2_1 >= 0.1',
 'x_r2_1 - x_r2_2 >= 0.1',
 'x_r2_2 - x_r2_3 >= 0.1']

### Run tests to solve for best possible score    

In [11]:
success = 0
fail_count = 0
while not success:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        success = 1
    except:
        fail_count += 1

print(f"Failed to get next trial {fail_count} times")

[WARNING 04-10 16:09:02] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 11.594851115749627, 'z_sh_1': 10.422259346240274, 'z_sh_2': 8.980727220980897, 'z_sh_3': 5.541771204409525, 'z_r2': 81.7920352315562, 'z_r2_1': 80.7153006753959, 'z_r2_2': 78.65855794480616, 'z_r2_3': 71.9845087215288, 'x_s': 0.85253869863227, 'x_s_1': 0.4612229035235942, 'x_s_2': -0.4853475414216518, 'x_s_3': -0.9167863462120295, 'x_r2': 0.8814975321292877, 'x_r2_1': 0.24527284856885667, 'x_r2_2': -0.10061240857467046, 'x_r2_3': -0.47467644540593035}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), Range

Failed to get next trial 56 times


In [12]:
parameterization

{'z_sh': 12,
 'z_sh_1': 10,
 'z_sh_2': 9,
 'z_sh_3': 6,
 'z_r2': 82,
 'z_r2_1': 81,
 'z_r2_2': 79,
 'z_r2_3': 72,
 'x_s': 0.85253869863227,
 'x_s_1': 0.4612229035235942,
 'x_s_2': -0.4853475414216518,
 'x_s_3': -0.9167863462120295,
 'x_r2': 0.8814975321292877,
 'x_r2_1': 0.24527284856885667,
 'x_r2_2': -0.10061240857467046,
 'x_r2_3': -0.47467644540593035}

In [13]:
print(len(parameterization))
for name, value in parameterization.items():
    print(f"{name}: {value}")

16
z_sh: 12
z_sh_1: 10
z_sh_2: 9
z_sh_3: 6
z_r2: 82
z_r2_1: 81
z_r2_2: 79
z_r2_3: 72
x_s: 0.85253869863227
x_s_1: 0.4612229035235942
x_s_2: -0.4853475414216518
x_s_3: -0.9167863462120295
x_r2: 0.8814975321292877
x_r2_1: 0.24527284856885667
x_r2_2: -0.10061240857467046
x_r2_3: -0.47467644540593035


In [14]:
Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True), add_gear = False))

[INFO 04-10 16:09:03] ax.service.ax_client: Completed trial 0 with data: {'Cross_term_gear_ratio_product': (np.float64(8872.782227), None)}.


In [15]:
list = param_to_list(parameterization, add_cl = True)
print(list)

[array([12, 10,  9,  6]), array([82, 81, 79, 72]), array([ 0.8525387 ,  0.4612229 , -0.48534754, -0.91678635]), array([ 0.88149753,  0.24527285, -0.10061241, -0.47467645]), array([0.0001, 0.0002, 0.0003, 0.0004])]


NOTE - run the following through 'Exhausted' errors.

In [16]:
init_count = 0
init_trials = 32 # hand set based on output at initialization
fail_count = 0
while init_count < init_trials:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        # Local evaluation here can be replaced with deployment to external system.
        Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True), add_gear = False))
    except:
        # print('Not this time')
        fail_count += 1
    init_count = trial_index
print(f'Failed {fail_count} times while using Sobol')

[WARNING 04-10 16:09:03] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 12.428504160018974, 'z_sh_1': 11.35012806500394, 'z_sh_2': 6.818484238920863, 'z_sh_3': 5.672312371779512, 'z_r2': 84.48225878143514, 'z_r2_1': 82.0620974805492, 'z_r2_2': 77.76834013907978, 'z_r2_3': 72.21701483219181, 'x_s': 0.051158969104290075, 'x_s_1': -0.08776422012597318, 'x_s_2': -0.38833082113414996, 'x_s_3': -0.8497417542152107, 'x_r2': 0.8889097991399466, 'x_r2_1': 0.688102522585541, 'x_r2_2': 0.1328910071402788, 'x_r2_3': -0.5897517647594214}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), Ra

Failed 693 times while using Sobol


In [17]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Cross_term_gear_ratio_product,z_sh,z_sh_1,z_sh_2,z_sh_3,...,z_r2_2,z_r2_3,x_s,x_s_1,x_s_2,x_s_3,x_r2,x_r2_1,x_r2_2,x_r2_3
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,8872.782227,12,10,9,6,...,79,72,0.852539,0.461223,-0.485348,-0.916786,0.881498,0.245273,-0.100612,-0.474676
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,9739.472591,12,11,7,6,...,78,72,0.051159,-0.087764,-0.388331,-0.849742,0.888910,0.688103,0.132891,-0.589752
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,14978.138531,12,11,9,7,...,71,70,0.308637,-0.116884,-0.352805,-0.637753,0.977737,0.387436,0.169031,-0.993066
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,9137.129494,12,11,9,6,...,78,72,0.979506,0.629408,-0.801876,-0.942703,0.993859,0.633158,0.448283,-0.354437
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,10759.507340,12,9,8,6,...,71,70,0.823046,0.457002,-0.101252,-0.911382,0.177348,0.012991,-0.336386,-0.942341
5,5,5_0,COMPLETED,Sobol,GenerationStep_0,15392.768516,12,11,8,6,...,77,75,0.458567,0.228272,-0.011724,-0.687003,0.628185,0.449645,-0.280487,-0.743190
6,6,6_0,COMPLETED,Sobol,GenerationStep_0,12264.600527,11,10,8,6,...,78,74,0.630323,0.013411,-0.109580,-0.815011,0.862837,0.733356,0.212547,-0.120198
7,7,7_0,COMPLETED,Sobol,GenerationStep_0,10876.219603,12,9,8,6,...,74,70,0.532841,0.342231,0.221892,-0.991905,0.909817,-0.024229,-0.276670,-0.806480
8,8,8_0,COMPLETED,Sobol,GenerationStep_0,10255.645631,12,10,8,7,...,78,71,0.681542,0.216553,0.014679,-0.741022,0.954079,0.173946,-0.149079,-0.499196
9,9,9_0,COMPLETED,Sobol,GenerationStep_0,17678.739456,11,10,8,7,...,73,72,0.896335,0.647011,0.490839,-0.033625,0.341547,0.171318,0.035437,-0.993843


In [18]:
render(Tag_client.get_optimization_trace())

In [19]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True), add_gear = False))

[INFO 04-10 16:14:00] ax.service.ax_client: Generated new trial 33 with parameters {'z_sh': 11, 'z_sh_1': 9, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 76, 'z_r2_1': 75, 'z_r2_2': 73, 'z_r2_3': 71, 'x_s': 0.767876, 'x_s_1': 0.667876, 'x_s_2': 0.567876, 'x_s_3': 0.017272, 'x_r2': -0.141004, 'x_r2_1': -0.241004, 'x_r2_2': -0.341004, 'x_r2_3': -0.441004} using model BoTorch.
[INFO 04-10 16:14:00] ax.service.ax_client: Completed trial 33 with data: {'Cross_term_gear_ratio_product': (np.float64(25835.734592), None)}.
[INFO 04-10 16:14:11] ax.service.ax_client: Generated new trial 34 with parameters {'z_sh': 11, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 75, 'z_r2_1': 74, 'z_r2_2': 73, 'z_r2_3': 72, 'x_s': 0.990792, 'x_s_1': 0.890792, 'x_s_2': 0.790792, 'x_s_3': 0.392181, 'x_r2': -0.127448, 'x_r2_1': -0.227448, 'x_r2_2': -0.327448, 'x_r2_3': -0.427448} using model BoTorch.
[INFO 04-10 16:14:11] ax.service.ax_client: Completed trial 34 with data: {'Cross_term_gear_ratio_product': (np.float64(26765

In [20]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")

z_sh: 9
z_sh_1: 8
z_sh_2: 7
z_sh_3: 6
z_r2: 75
z_r2_1: 74
z_r2_2: 73
z_r2_3: 72
x_s: 1.0
x_s_1: 0.8999999999998373
x_s_2: 0.7999999999988541
x_s_3: -0.999999999999853
x_r2: 0.9999999999985899
x_r2_1: 0.719421838988003
x_r2_2: 0.6194218389874829
x_r2_3: 0.5194218389853328


In [21]:
print(score_vals(param_to_list(best_parameters, add_cl = True), add_gear = False))

35727.0877139389


In [25]:
param_to_list(best_parameters, add_cl = True)

[array([9, 8, 7, 6]),
 array([75, 74, 73, 72]),
 array([ 1. ,  0.9,  0.8, -1. ]),
 array([1.        , 0.71942184, 0.61942184, 0.51942184]),
 array([0.0001, 0.0002, 0.0003, 0.0004])]

In [26]:
render(Tag_client.get_optimization_trace())

In [24]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-10 16:38:47] ax.service.ax_client: Generated new trial 133 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 75, 'z_r2_1': 74, 'z_r2_2': 73, 'z_r2_3': 72, 'x_s': 1.0, 'x_s_1': 0.9, 'x_s_2': 0.8, 'x_s_3': -1.0, 'x_r2': 1.0, 'x_r2_1': 0.653153, 'x_r2_2': 0.553153, 'x_r2_3': 0.453153} using model BoTorch.
[INFO 04-10 16:38:47] ax.service.ax_client: Completed trial 133 with data: {'Cross_term_gear_ratio_product': (np.float64(2929.941848), None)}.
[INFO 04-10 16:38:56] ax.service.ax_client: Generated new trial 134 with parameters {'z_sh': 10, 'z_sh_1': 9, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 76, 'z_r2_1': 75, 'z_r2_2': 74, 'z_r2_3': 72, 'x_s': 1.0, 'x_s_1': -0.8, 'x_s_2': -0.9, 'x_s_3': -1.0, 'x_r2': 1.0, 'x_r2_1': 0.9, 'x_r2_2': 0.8, 'x_r2_3': 0.7} using model BoTorch.
[INFO 04-10 16:38:56] ax.service.ax_client: Completed trial 134 with data: {'Cross_term_gear_ratio_product': (np.float64(2689.129012), None)}.
[INFO 04-10 16:39:07] ax.service.ax_client: Genera

KeyboardInterrupt: 

In [27]:
render(Tag_client.get_optimization_trace())

In [ ]:
for i in range(800):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-10 16:51:51] ax.service.ax_client: Generated new trial 183 with parameters {'z_sh': 11, 'z_sh_1': 9, 'z_sh_2': 8, 'z_sh_3': 6, 'z_r2': 86, 'z_r2_1': 73, 'z_r2_2': 72, 'z_r2_3': 71, 'x_s': 1.0, 'x_s_1': 0.241947, 'x_s_2': 0.141909, 'x_s_3': -1.0, 'x_r2': 0.28209, 'x_r2_1': 0.18209, 'x_r2_2': 0.08209, 'x_r2_3': -0.678833} using model BoTorch.
[INFO 04-10 16:51:51] ax.service.ax_client: Completed trial 183 with data: {'Cross_term_gear_ratio_product': (np.float64(1083.555733), None)}.
[INFO 04-10 16:52:08] ax.service.ax_client: Generated new trial 184 with parameters {'z_sh': 11, 'z_sh_1': 9, 'z_sh_2': 8, 'z_sh_3': 6, 'z_r2': 75, 'z_r2_1': 74, 'z_r2_2': 72, 'z_r2_3': 71, 'x_s': 1.0, 'x_s_1': 0.9, 'x_s_2': 0.8, 'x_s_3': -0.16072, 'x_r2': 1.0, 'x_r2_1': -0.217613, 'x_r2_2': -0.317613, 'x_r2_3': -0.417613} using model BoTorch.
[INFO 04-10 16:52:08] ax.service.ax_client: Completed trial 184 with data: {'Cross_term_gear_ratio_product': (np.float64(2487.311592), None)}.
[INFO 04-10 16:5